# Ollama server

In [30]:
%pip -q install python-magic tabulate pdf2image pytesseract llama-index llama-index-readers-web llama-index-llms-ollama llama-index-embeddings-huggingface llama-index-readers-file unstructured einops datasets huggingface_hub sentence-transformers docx2txt python-docx

In [31]:
! curl https://ollama.ai/install.sh | sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 10941    0 10941    0     0  52381      0 --:--:-- --:--:-- --:--:-- 52349>>> Downloading ollama...
100 10941    0 10941    0     0  38460      0 --:--:-- --:--:-- --:--:-- 38389
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [32]:
import os
import threading
import subprocess
import requests
import json

def ollama():
    os.environ['OLLAMA_HOST'] = '0.0.0.0:11434'
    os.environ['OLLAMA_ORIGINS'] = '*'
    subprocess.Popen(["ollama", "serve"])


In [33]:
ollama_thread = threading.Thread(target=ollama)
ollama_thread.start()

In [34]:
!ollama pull llama3

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest 
pulling 6a0746a1ec1a... 100% ▕▏ 4.7 GB                         
pulling 4fa551d4f938... 100% ▕▏  12 KB                         
pulling 8ab4849b038c... 100% ▕▏  254 B                         
pulling 577073ffcc6c... 100% ▕▏  110 B                         
pulling 3f8eb4da87fa... 100% ▕▏  485 B                         
verifying sha256 digest 
writing manifest 
removing any unused layers 
success 


In [35]:
!ollama list

NAME         	ID          	SIZE  	MODIFIED      
llama3:latest	365c0bd3c000	4.7 GB	3 seconds ago	


# Libraries

In [36]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core import VectorStoreIndex
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings
from llama_index.core import get_response_synthesizer
from llama_index.core import DocumentSummaryIndex
from llama_index.core.node_parser import SentenceSplitter
from llama_index.readers.web import SimpleWebPageReader
from llama_index.readers.file import (
    DocxReader,
    HWPReader,
    PDFReader,
    EpubReader,
    FlatReader,
    HTMLTagReader,
    ImageCaptionReader,
    ImageReader,
    ImageVisionLLMReader,
    IPYNBReader,
    MarkdownReader,
    MboxReader,
    PptxReader,
    PandasCSVReader,
    VideoAudioReader,
    UnstructuredReader,
    PyMuPDFReader,
    ImageTabularChartReader,
    XMLReader,
    PagedCSVReader,
    CSVReader,
    RTFReader,
)
from llama_index.core import (
    load_index_from_storage,
    StorageContext,
)


# Trying Embedding models from MTEB

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
!huggingface-cli whoami

ballav


# If want to download the model locally

In [ ]:
# !huggingface-cli download meta-llama/Meta-Llama-3-8B --include "original/*" --local-dir Meta-Llama-3-8B
!huggingface-cli download meta-llama/Meta-Llama-3-8B --local-dir Meta-Llama-3-8B


Fetching 17 files:   0% 0/17 [00:00<?, ?it/s]Downloading 'README.md' to 'Meta-Llama-3-8B/.huggingface/download/README.md.53e7e6ec18e5d060bc5690314c0e04f2df3326bf.incomplete'

README.md: 100% 36.5k/36.5k [00:00<00:00, 5.15MB/s]
Download complete. Moving file to Meta-Llama-3-8B/README.md

USE_POLICY.md: 100% 4.70k/4.70k [00:00<00:00, 18.1MB/s]
Download complete. Moving file to Meta-Llama-3-8B/USE_POLICY.md

LICENSE: 100% 7.80k/7.80k [00:00<00:00, 34.2MB/s]
Download complete. Moving file to Meta-Llama-3-8B/LICENSE

config.json: 100% 654/654 [00:00<00:00, 5.30MB/s]
Download complete. Moving file to Meta-Llama-3-8B/config.json

model-00002-of-00004.safetensors:   0% 0.00/5.00G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0% 0.00/4.98G [00:00<?, ?B/s]


generation_config.json: 100% 177/177 [00:00<00:00, 1.23MB/s]
Download complete. Moving file to Meta-Llama-3-8B/generation_config.json



model-00003-of-00004.safetensors:   0% 0.00/4.92G [00:00<?, ?B/s]



model.safetensors.index.json

In [ ]:
Settings.embed_model = HuggingFaceEmbedding(model_name="nvidia/NV-Embed-v1", trust_remote_code=True, use_auth_token=True)

/usr/local/lib/python3.10/dist-packages/sentence_transformers/SentenceTransformer.py:99: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v3 of SentenceTransformers.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
from transformers import LlamaForCausalLM, LlamaTokenizer

model = LlamaForCausalLM.from_pretrained("/content/Meta-Llama-3-8B(1)")
tokenizer = LlamaTokenizer.from_pretrained("/content/Meta-Llama-3-8B(1)")


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

# Data loading and storing indexes

# 1. Tablular data from webpage




*   Detecting the tables
*   Saving it as csv
*   Mapping the csv files with their corresponding url
*   Indexing of these csv data



In [37]:
file_to_url_mapping={}

In [38]:
def table_to_csv_converter(url,file_to_url_mapping):

  from bs4 import BeautifulSoup
  import requests
  import pandas as pd
  import os

  url = url
  page = requests.get(url)
  soup = BeautifulSoup(page.text, 'html')

  # Detect all tables on the webpage
  all_tables = soup.find_all('table')
  print(f"Total number of tables: {len(all_tables)}")

  # Create a directory to save the CSV files
  directory = 'tabular_data_csv'
  if not os.path.exists(directory):
      os.makedirs(directory)

  # Iterate over each table
  for i, table in enumerate(all_tables):
      # Get the caption of the table
      table_caption = table.find('caption').text.strip()

      world_titles = table.find_all('th')
      world_table_titles = [title.text.strip() for title in world_titles]

      df = pd.DataFrame(columns=world_table_titles)

      column_data = table.find_all('tr')
      for row in column_data[1:]:
          row_data = row.find_all('td')
          individual_row_data = [data.text.strip() for data in row_data]
          df.loc[len(df)] = individual_row_data

      # Save the DataFrame as a CSV file with the table caption as the file name
      csv_file = os.path.join(directory, f'{table_caption}.csv')
      df.to_csv(csv_file, index=False)
      print(f"CSV file saved: {csv_file}")
      #file_to_url_mapping.append({ f'{table_caption}.csv':url})
      file_to_url_mapping[f'{table_caption}.csv'] = url

In [39]:
url_with_table = ['https://maharashtra.nic.in/',
 'https://maharashtra.nic.in/services/',
 'https://maharashtra.nic.in/profile/',
 'https://www.nic.in/servicecontents/nicnet/',
 'https://www.nic.in/servicecontents/data-centre/',
 'https://www.nic.in/servicecontents/national-cloud/',
 'https://www.nic.in/servicecontents/messaging/',
 'https://www.nic.in/servicecontents/remote-sensing-gis/',
 'https://www.nic.in/servicecontents/webcast/',
 'https://www.nic.in/servicecontents/nkn/',
 'https://www.nic.in/servicecontents/command-and-control-centre/',
 'https://www.nic.in/servicecontents/government-local-area-networks-lans/',
 'https://www.nic.in/servicecontents/video-conferencing/',
 'https://www.nic.in/servicecontents/security/',
 'https://www.nic.in/servicecontents/centralised-aadhaar-vault/',
 'https://maharashtra.nic.in/infrastructure/',
 'https://maharashtra.nic.in/news-update/',
 'https://maharashtra.nic.in/events/',
 'https://maharashtra.nic.in/awards/',
 'https://www.nic.in/servicecontents/domain-registration/',
 'https://maharashtra.nic.in/contact-us/',
 'https://maharashtra.nic.in/website-policies/',
 'https://maharashtra.nic.in/help/',
 'https://maharashtra.nic.in/web-information-manager/',
 'https://maharashtra.nic.in/directory/',
 'https://maharashtra.nic.in/rti/',
 'https://maharashtra.nic.in/district-centres/',
 'https://maharashtra.nic.in/photo-gallery/',
 'https://maharashtra.nic.in/video-gallery/', 'https://maharashtra.nic.in/organization-structure/']

In [40]:
for url in url_with_table:
  table_to_csv_converter(url,file_to_url_mapping)

Total number of tables: 0
Total number of tables: 0
Total number of tables: 0
Total number of tables: 0
Total number of tables: 0
Total number of tables: 0
Total number of tables: 0
Total number of tables: 0
Total number of tables: 0
Total number of tables: 0
Total number of tables: 0
Total number of tables: 0
Total number of tables: 0
Total number of tables: 0
Total number of tables: 0
Total number of tables: 0
Total number of tables: 0
Total number of tables: 0
Total number of tables: 0
Total number of tables: 0
Total number of tables: 0
Total number of tables: 0
Total number of tables: 2
CSV file saved: tabular_data_csv/Screen Reader Information.csv
CSV file saved: tabular_data_csv/Plug-in for alternate document types.csv
Total number of tables: 0
Total number of tables: 37
CSV file saved: tabular_data_csv/SIO.csv
CSV file saved: tabular_data_csv/Maharashtra State Centre, Mumbai.csv
CSV file saved: tabular_data_csv/Ahmadnagar.csv
CSV file saved: tabular_data_csv/Akola.csv
CSV file s

In [41]:
file_to_url_mapping

{'Screen Reader Information.csv': 'https://maharashtra.nic.in/help/',
 'Plug-in for alternate document types.csv': 'https://maharashtra.nic.in/help/',
 'SIO.csv': 'https://maharashtra.nic.in/directory/',
 'Maharashtra State Centre, Mumbai.csv': 'https://maharashtra.nic.in/directory/',
 'Ahmadnagar.csv': 'https://maharashtra.nic.in/directory/',
 'Akola.csv': 'https://maharashtra.nic.in/directory/',
 'Amravati.csv': 'https://maharashtra.nic.in/directory/',
 'Aurangabad.csv': 'https://maharashtra.nic.in/directory/',
 'Beed.csv': 'https://maharashtra.nic.in/directory/',
 'Bhandara.csv': 'https://maharashtra.nic.in/directory/',
 'Buldana.csv': 'https://maharashtra.nic.in/directory/',
 'Chandrapur.csv': 'https://maharashtra.nic.in/directory/',
 'Dhule.csv': 'https://maharashtra.nic.in/directory/',
 'Gadchiroli.csv': 'https://maharashtra.nic.in/directory/',
 'Gondia.csv': 'https://maharashtra.nic.in/directory/',
 'Hingoli.csv': 'https://maharashtra.nic.in/directory/',
 'Jalgaon.csv': 'https:/

# CSV index

In [42]:
from llama_index.core import VectorStoreIndex, SummaryIndex
from llama_index.readers.web import SimpleWebPageReader
import os
from llama_index.core import (
    load_index_from_storage,
    StorageContext,
)

Settings.llm = Ollama(model="llama3", temperature=0, request_timeout=500.0)

# # Overall embedding (due to size not able to load it)
# Settings.embed_model = HuggingFaceEmbedding(model_name="nvidia/NV-Embed-v1", trust_remote_code=True, use_auth_token=True)

# # Embedding used earlier
Settings.embed_model = HuggingFaceEmbedding(model_name="Alibaba-NLP/gte-large-en-v1.5", trust_remote_code=True)


# Paged CSV Reader example
parser = PagedCSVReader()

file_extractor = {".csv": parser}  # Add other CSV formats as needed
documents = SimpleDirectoryReader(
    "./tabular_data_csv", file_extractor=file_extractor
).load_data()
documents

# Create and save the Indexes for the loaded data
index = VectorStoreIndex.from_documents(documents, llm=Settings.llm, embed_model=Settings.embed_model)



#  save index1 to disk
index.set_index_id("vector_index")
index.storage_context.persist("./tabular_data_csv_index")






/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


# 2. Text data from the webpages

In [59]:
# all the urls

urls = {
    "https://maharashtra.nic.in/": "Home_index",
    "https://maharashtra.nic.in/services/": "Services_index",
    "https://maharashtra.nic.in/profile/": "profile_index",
    "https://www.nic.in/servicecontents/nicnet/": "nicnet_index",
    "https://www.nic.in/servicecontents/data-centre/": "data-centre_index",
    "https://www.nic.in/servicecontents/national-cloud/": "national-cloud_index",
    "https://www.nic.in/servicecontents/messaging/": "messaging_index",
    "https://www.nic.in/servicecontents/remote-sensing-gis/": "remote-sensing-gis_index",
    "https://www.nic.in/servicecontents/webcast/": "webcast_index",
    "https://www.nic.in/servicecontents/nkn/": "nkn_index",
    "https://www.nic.in/servicecontents/command-and-control-centre/": "command-and-control-centre_index",
    "https://www.nic.in/servicecontents/government-local-area-networks-lans/": "government-local-area-networks-lans_index",
    "https://www.nic.in/servicecontents/video-conferencing/": "video-conferencing_index",
    "https://www.nic.in/servicecontents/security/": "security_index",
    "https://www.nic.in/servicecontents/centralised-aadhaar-vault/": "centralised-aadhaar-vault_index",
    "https://maharashtra.nic.in/infrastructure/": "infrastructure_index",
    "https://maharashtra.nic.in/news-update/": "news-update_index",
    "https://maharashtra.nic.in/events/": "events_index",
    "https://maharashtra.nic.in/awards/": "awards_index",
    "https://www.nic.in/servicecontents/domain-registration/": "domain_registration_index",
    "https://maharashtra.nic.in/contact-us/": "contact-us_index",
    "https://maharashtra.nic.in/website-policies/": "website-policies_index",
    "https://maharashtra.nic.in/help/": "help_index",
    "https://maharashtra.nic.in/web-information-manager/": "web-information-manager_index",
    "https://maharashtra.nic.in/directory/": "directory_index",
    "https://maharashtra.nic.in/rti/": "rti_index",
    "https://maharashtra.nic.in/district-centres/": "district-centres_index",
    "https://maharashtra.nic.in/photo-gallery/": "photo-gallery_index",
    "https://maharashtra.nic.in/video-gallery/": "video-gallery_index",
    "https://maharashtra.nic.in/organization-structure/": "organization-structure_index"
}


In [44]:
# URLs and corresponding link names
urls = {
    "https://maharashtra.nic.in/": "Home_index",
    "https://maharashtra.nic.in/services/": "Services_index",
    "https://maharashtra.nic.in/profile/": "profile_index",
    "https://www.nic.in/servicecontents/nicnet/": "nicnet_index",
    "https://www.nic.in/servicecontents/data-centre/": "data-centre_index",
    "https://www.nic.in/servicecontents/national-cloud/": "national-cloud_index",
    "https://www.nic.in/servicecontents/messaging/": "messaging_index",
    "https://www.nic.in/servicecontents/remote-sensing-gis/": "remote-sensing-gis_index",
    "https://www.nic.in/servicecontents/webcast/": "webcast_index",
    "https://www.nic.in/servicecontents/nkn/": "nkn_index",
    "https://www.nic.in/servicecontents/command-and-control-centre/": "command-and-control-centre_index",
    "https://www.nic.in/servicecontents/government-local-area-networks-lans/": "government-local-area-networks-lans_index",
    "https://www.nic.in/servicecontents/video-conferencing/": "video-conferencing_index",
    "https://www.nic.in/servicecontents/security/": "security_index",
    "https://www.nic.in/servicecontents/centralised-aadhaar-vault/": "centralised-aadhaar-vault_index",
    "https://maharashtra.nic.in/infrastructure/": "infrastructure_index",
    "https://maharashtra.nic.in/awards/": "awards_index",
    "https://www.nic.in/servicecontents/domain-registration/": "domain_registration_index",
    "https://maharashtra.nic.in/contact-us/": "contact-us_index",
    "https://maharashtra.nic.in/website-policies/": "website-policies_index",
    "https://maharashtra.nic.in/help/": "help_index",
    "https://maharashtra.nic.in/web-information-manager/": "web-information-manager_index",
    "https://maharashtra.nic.in/rti/": "rti_index",
    "https://maharashtra.nic.in/video-gallery/": "video-gallery_index"
}

In [45]:
from llama_index.core import VectorStoreIndex, SummaryIndex
from llama_index.readers.web import SimpleWebPageReader
import os
from llama_index.core import (
    SimpleDirectoryReader,
    load_index_from_storage,
    StorageContext,
)
from llama_index.core import download_loader
from docx import Document


# Set the configurations for the models
Settings.llm = Ollama(model="llama3", temperature=0, request_timeout=500.0)
Settings.embed_model = HuggingFaceEmbedding(model_name="Alibaba-NLP/gte-large-en-v1.5", trust_remote_code=True)


# Define the directory path to save the documents and indexes
save_directory = "/content/webpages_doc"

if not os.path.exists(save_directory):
    os.makedirs(save_directory)


# Load data from the specified URLs and save each document with the link name
BeautifulSoupWebReader = download_loader("BeautifulSoupWebReader")
loader = BeautifulSoupWebReader()

for url, link_name in urls.items():
    documents = loader.load_data(urls=[url])
    print(f"Loaded {len(documents)} documents of type {type(documents[0])} from {url}")

    file_path = os.path.join(save_directory, f"{link_name}.docx")
    doc = Document()
    for document in documents:
        doc.add_paragraph(document.text)
    doc.save(file_path)

    print(f"Data from {url} saved successfully to: {file_path}")

<ipython-input-45-fca3f8c31a2f>:26: DeprecationWarning: Call to deprecated function (or staticmethod) download_loader. (`download_loader()` is deprecated. Please install tool using pip install directly instead.)
  BeautifulSoupWebReader = download_loader("BeautifulSoupWebReader")


Loaded 1 documents of type <class 'llama_index.core.schema.Document'> from https://maharashtra.nic.in/
Data from https://maharashtra.nic.in/ saved successfully to: /content/webpages_doc/Home_index.docx
Loaded 1 documents of type <class 'llama_index.core.schema.Document'> from https://maharashtra.nic.in/services/
Data from https://maharashtra.nic.in/services/ saved successfully to: /content/webpages_doc/Services_index.docx
Loaded 1 documents of type <class 'llama_index.core.schema.Document'> from https://maharashtra.nic.in/profile/
Data from https://maharashtra.nic.in/profile/ saved successfully to: /content/webpages_doc/profile_index.docx
Loaded 1 documents of type <class 'llama_index.core.schema.Document'> from https://www.nic.in/servicecontents/nicnet/
Data from https://www.nic.in/servicecontents/nicnet/ saved successfully to: /content/webpages_doc/nicnet_index.docx
Loaded 1 documents of type <class 'llama_index.core.schema.Document'> from https://www.nic.in/servicecontents/data-cent

In [25]:
from llama_index.core import VectorStoreIndex, SummaryIndex
from llama_index.readers.web import SimpleWebPageReader
import os
from llama_index.core import (
    load_index_from_storage,
    StorageContext,
)

Settings.llm = Ollama(model="llama3", temperature=0, request_timeout=500.0)
# # Embedding used earlier
Settings.embed_model = HuggingFaceEmbedding(model_name="Alibaba-NLP/gte-large-en-v1.5", trust_remote_code=True)
# Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5", trust_remote_code=True)

# Paged CSV Reader example
parser = DocxReader()

file_extractor = {".docx": parser}
documents = SimpleDirectoryReader(
    "./webpages_doc", file_extractor=file_extractor
).load_data()
documents

# Create and save the Indexes for the loaded data
index = VectorStoreIndex.from_documents(documents, llm=Settings.llm, embed_model=Settings.embed_model)



#  save index1 to disk
index.set_index_id("vector_index")
index.storage_context.persist("./webpages_doc_index")






# Combined Index (both csv and docx)

In [46]:
from llama_index.core import VectorStoreIndex, SummaryIndex
from llama_index.readers.web import SimpleWebPageReader
import os
from llama_index.core import (
    load_index_from_storage,
    StorageContext,
)
from llama_index.readers.file import DocxReader, PagedCSVReader

# Set up the language model and embedding
Settings.llm = Ollama(model="llama3", temperature=0, request_timeout=500.0)
Settings.embed_model = HuggingFaceEmbedding(model_name="Alibaba-NLP/gte-large-en-v1.5", trust_remote_code=True)

# Readers for different file types
docx_parser = DocxReader()
csv_parser = PagedCSVReader()

file_extractor = {
    ".docx": docx_parser,
    ".csv": csv_parser
}

# Load documents from both directories
docx_documents = SimpleDirectoryReader("./webpages_doc", file_extractor=file_extractor).load_data()
csv_documents = SimpleDirectoryReader("./tabular_data_csv", file_extractor=file_extractor).load_data()

# Combine the documents from both sources
all_documents = docx_documents + csv_documents

# Create and save the Indexes for the loaded data
index = VectorStoreIndex.from_documents(all_documents, llm=Settings.llm, embed_model=Settings.embed_model)

# Save the combined index to disk
index.set_index_id("combined_vector_index")
index.storage_context.persist("./combined_data_index")

# Optionally, to verify loading the index:
# storage_context = StorageContext.from_directory("./combined_data_index")
# loaded_index = load_index_from_storage(storage_context, index_id="combined_vector_index")


# Creating a zip of colab folder

In [57]:
import shutil
import os
import zipfile

# Specify the parent folder path
parent_folder = '/content/combined_data_index'

# Create a zip file
zip_file = zipfile.ZipFile('combined_data_index_v2.zip', 'w')

# Iterate over the parent folder and its subfolders
for root, dirs, files in os.walk(parent_folder):
    for file in files:
        file_path = os.path.join(root, file)
        zip_file.write(file_path, os.path.relpath(file_path, parent_folder))

zip_file.close()


In [58]:
from google.colab import files
files.download("combined_data_index_v2.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# extracting folder paths

In [ ]:
import os

# Specify the path to the parent directory
parent_dir = '/content/Index'

# Initialize an empty list to store the folder paths
folder_paths = []

# Iterate over the contents of the parent directory
for item in os.listdir(parent_dir):
    # Check if the item is a directory
    if os.path.isdir(os.path.join(parent_dir, item)):
        # Add the full path of the directory to the list
        folder_paths.append(os.path.join(parent_dir, item))

# Print the list of folder paths
print(folder_paths)
print(len(folder_paths))

['/content/Index/Services_index', '/content/Index/rti_index', '/content/Index/webcast_index', '/content/Index/nicnet_index', '/content/Index/messaging_index', '/content/Index/help_index', '/content/Index/Home_index', '/content/Index/infrastructure_index', '/content/Index/website-policies_index', '/content/Index/profile_index', '/content/Index/domain_registration_index', '/content/Index/nkn_index', '/content/Index/web-information-manager_index', '/content/Index/government-local-area-networks-lans_index', '/content/Index/directory_index', '/content/Index/tabular_data_csv_index', '/content/Index/awards_index', '/content/Index/centralised-aadhaar-vault_index', '/content/Index/photo-gallery_index', '/content/Index/contact-us_index', '/content/Index/data-centre_index', '/content/Index/events_index', '/content/Index/remote-sensing-gis_index', '/content/Index/news-update_index', '/content/Index/video-conferencing_index', '/content/Index/security_index', '/content/Index/national-cloud_index', '

# Combining index

In [ ]:
import os
import json

# List of folders
# folders = ['/content/Index/video-gallery_index', '/content/Index/remote-sensing-gis_index', '/content/Index/help_index', '/content/Index/events_index', '/content/Index/Services_index', '/content/Index/domain_registration_index', '/content/Index/data-centre_index', '/content/Index/messaging_index', '/content/Index/web-information-manager_index', '/content/Index/nicnet_index', '/content/Index/website-policies_index', '/content/Index/news-update_index', '/content/Index/government-local-area-networks-lans_index', '/content/Index/profile_index', '/content/Index/webcast_index', '/content/Index/rti_index', '/content/Index/nkn_index', '/content/Index/awards_index', '/content/Index/security_index', '/content/Index/photo-gallery_index', '/content/Index/centralised-aadhaar-vault_index', '/content/Index/Home_index', '/content/Index/contact-us_index', '/content/Index/infrastructure_index', '/content/Index/video-conferencing_index', '/content/Index/command-and-control-centre_index', '/content/Index/national-cloud_index', '/content/Index/directory_index', '/content/tabular_data_csv_index', '/content/text_Index']
# folders = ['/content/Index/video-gallery_index', '/content/Index/remote-sensing-gis_index', '/content/Index/help_index', '/content/Index/events_index', '/content/Index/Services_index', '/content/Index/domain_registration_index', '/content/Index/data-centre_index', '/content/Index/messaging_index', '/content/Index/web-information-manager_index', '/content/Index/nicnet_index', '/content/Index/website-policies_index', '/content/Index/news-update_index', '/content/Index/government-local-area-networks-lans_index', '/content/Index/profile_index', '/content/Index/webcast_index', '/content/Index/rti_index', '/content/Index/nkn_index', '/content/Index/awards_index', '/content/Index/security_index', '/content/Index/photo-gallery_index', '/content/Index/centralised-aadhaar-vault_index', '/content/Index/Home_index', '/content/Index/contact-us_index', '/content/Index/infrastructure_index', '/content/Index/video-conferencing_index', '/content/Index/command-and-control-centre_index', '/content/Index/national-cloud_index', '/content/Index/directory_index', '/content/tabular_data_csv_index', '/content/Text_Index_v1/news_updates', '/content/Text_Index_v1/services', '/content/Text_Index_v1/districts', '/content/Text_Index_v1/events', '/content/Text_Index_v1/photo_gallery', '/content/Text_Index_v1/Nic']

folders = ['/content/Index/Services_index', '/content/Index/rti_index', '/content/Index/webcast_index', '/content/Index/nicnet_index', '/content/Index/messaging_index', '/content/Index/help_index', '/content/Index/Home_index', '/content/Index/infrastructure_index', '/content/Index/website-policies_index', '/content/Index/profile_index', '/content/Index/domain_registration_index', '/content/Index/nkn_index', '/content/Index/web-information-manager_index', '/content/Index/government-local-area-networks-lans_index', '/content/Index/directory_index', '/content/Index/tabular_data_csv_index', '/content/Index/awards_index', '/content/Index/centralised-aadhaar-vault_index', '/content/Index/photo-gallery_index', '/content/Index/contact-us_index', '/content/Index/data-centre_index', '/content/Index/events_index', '/content/Index/remote-sensing-gis_index', '/content/Index/news-update_index', '/content/Index/video-conferencing_index', '/content/Index/security_index', '/content/Index/national-cloud_index', '/content/Index/video-gallery_index', '/content/Index/docs_index', '/content/Index/command-and-control-centre_index']


# Initialize merged data
merged_data = {
    "embedding_dict": {},
    "text_id_to_ref_doc_id": {},
    "metadata_dict": {}
}

# Loop through each folder
for folder in folders:
    file_path = os.path.join(folder, 'default__vector_store.json')

    # Load the contents of the JSON file
    with open(file_path, 'r') as file:
        data = json.load(file)

    # Merge the embedding_dict
    merged_data['embedding_dict'] = {**merged_data['embedding_dict'], **data['embedding_dict']}

    # Merge the text_id_to_ref_doc_id
    merged_data['text_id_to_ref_doc_id'] = {**merged_data['text_id_to_ref_doc_id'], **data['text_id_to_ref_doc_id']}

    # Merge the metadata_dict
    merged_data['metadata_dict'] = {**merged_data['metadata_dict'], **data['metadata_dict']}

# Write the merged data to a new JSON file
with open('/content/combined_index_v4/default__vector_store.json', 'w') as merged_file:
    json.dump(merged_data, merged_file, indent=4)

In [ ]:
import os
import json

# List of folders
# folders = ['/content/Index/video-gallery_index', '/content/Index/remote-sensing-gis_index', '/content/Index/help_index', '/content/Index/events_index', '/content/Index/Services_index', '/content/Index/domain_registration_index', '/content/Index/data-centre_index', '/content/Index/messaging_index', '/content/Index/web-information-manager_index', '/content/Index/nicnet_index', '/content/Index/website-policies_index', '/content/Index/news-update_index', '/content/Index/government-local-area-networks-lans_index', '/content/Index/profile_index', '/content/Index/webcast_index', '/content/Index/rti_index', '/content/Index/nkn_index', '/content/Index/awards_index', '/content/Index/security_index', '/content/Index/photo-gallery_index', '/content/Index/centralised-aadhaar-vault_index', '/content/Index/Home_index', '/content/Index/contact-us_index', '/content/Index/infrastructure_index', '/content/Index/video-conferencing_index', '/content/Index/command-and-control-centre_index', '/content/Index/national-cloud_index', '/content/Index/directory_index', '/content/tabular_data_csv_index', '/content/text_Index']
# folders = ['/content/Index/video-gallery_index', '/content/Index/remote-sensing-gis_index', '/content/Index/help_index', '/content/Index/events_index', '/content/Index/Services_index', '/content/Index/domain_registration_index', '/content/Index/data-centre_index', '/content/Index/messaging_index', '/content/Index/web-information-manager_index', '/content/Index/nicnet_index', '/content/Index/website-policies_index', '/content/Index/news-update_index', '/content/Index/government-local-area-networks-lans_index', '/content/Index/profile_index', '/content/Index/webcast_index', '/content/Index/rti_index', '/content/Index/nkn_index', '/content/Index/awards_index', '/content/Index/security_index', '/content/Index/photo-gallery_index', '/content/Index/centralised-aadhaar-vault_index', '/content/Index/Home_index', '/content/Index/contact-us_index', '/content/Index/infrastructure_index', '/content/Index/video-conferencing_index', '/content/Index/command-and-control-centre_index', '/content/Index/national-cloud_index', '/content/Index/directory_index', '/content/tabular_data_csv_index', '/content/Text_Index_v1/news_updates', '/content/Text_Index_v1/services', '/content/Text_Index_v1/districts', '/content/Text_Index_v1/events', '/content/Text_Index_v1/photo_gallery', '/content/Text_Index_v1/Nic']

folders = ['/content/Index/Services_index', '/content/Index/rti_index', '/content/Index/webcast_index', '/content/Index/nicnet_index', '/content/Index/messaging_index', '/content/Index/help_index', '/content/Index/Home_index', '/content/Index/infrastructure_index', '/content/Index/website-policies_index', '/content/Index/profile_index', '/content/Index/domain_registration_index', '/content/Index/nkn_index', '/content/Index/web-information-manager_index', '/content/Index/government-local-area-networks-lans_index', '/content/Index/directory_index', '/content/Index/tabular_data_csv_index', '/content/Index/awards_index', '/content/Index/centralised-aadhaar-vault_index', '/content/Index/photo-gallery_index', '/content/Index/contact-us_index', '/content/Index/data-centre_index', '/content/Index/events_index', '/content/Index/remote-sensing-gis_index', '/content/Index/news-update_index', '/content/Index/video-conferencing_index', '/content/Index/security_index', '/content/Index/national-cloud_index', '/content/Index/video-gallery_index', '/content/Index/docs_index', '/content/Index/command-and-control-centre_index']


# Initialize merged data
merged_data = {
    "docstore/metadata": {},
    "docstore/data": {},
    "docstore/ref_doc_info": {}
}

# Loop through each folder
for folder in folders:
    file_path = os.path.join(folder, 'docstore.json')

    # Load the contents of the JSON file
    with open(file_path, 'r') as file:
        data = json.load(file)

    # Merge the docstore/metadata
    merged_data['docstore/metadata'] = {**merged_data['docstore/metadata'], **data['docstore/metadata']}

    # Merge the docstore/data
    merged_data['docstore/data'] = {**merged_data['docstore/data'], **data['docstore/data']}

    # Merge the docstore/ref_doc_info
    merged_data['docstore/ref_doc_info'] = {**merged_data['docstore/ref_doc_info'], **data['docstore/ref_doc_info']}

# Create the merged JSON data
merged_data_final = {
    "docstore/metadata": merged_data['docstore/metadata'],
    "docstore/data": merged_data['docstore/data'],
    "docstore/ref_doc_info": merged_data['docstore/ref_doc_info']
}

# Write the merged data to a new JSON file
with open('/content/combined_index_v4/docstore.json', 'w') as merged_file:
    json.dump(merged_data_final, merged_file, indent=4)

In [ ]:
import os
import json

# List of folders
# folders = ['/content/Index/video-gallery_index', '/content/Index/remote-sensing-gis_index', '/content/Index/help_index', '/content/Index/events_index', '/content/Index/Services_index', '/content/Index/domain_registration_index', '/content/Index/data-centre_index', '/content/Index/messaging_index', '/content/Index/web-information-manager_index', '/content/Index/nicnet_index', '/content/Index/website-policies_index', '/content/Index/news-update_index', '/content/Index/government-local-area-networks-lans_index', '/content/Index/profile_index', '/content/Index/webcast_index', '/content/Index/rti_index', '/content/Index/nkn_index', '/content/Index/awards_index', '/content/Index/security_index', '/content/Index/photo-gallery_index', '/content/Index/centralised-aadhaar-vault_index', '/content/Index/Home_index', '/content/Index/contact-us_index', '/content/Index/infrastructure_index', '/content/Index/video-conferencing_index', '/content/Index/command-and-control-centre_index', '/content/Index/national-cloud_index', '/content/Index/directory_index', '/content/tabular_data_csv_index', '/content/text_Index']
# folders = ['/content/Index/video-gallery_index', '/content/Index/remote-sensing-gis_index', '/content/Index/help_index', '/content/Index/events_index', '/content/Index/Services_index', '/content/Index/domain_registration_index', '/content/Index/data-centre_index', '/content/Index/messaging_index', '/content/Index/web-information-manager_index', '/content/Index/nicnet_index', '/content/Index/website-policies_index', '/content/Index/news-update_index', '/content/Index/government-local-area-networks-lans_index', '/content/Index/profile_index', '/content/Index/webcast_index', '/content/Index/rti_index', '/content/Index/nkn_index', '/content/Index/awards_index', '/content/Index/security_index', '/content/Index/photo-gallery_index', '/content/Index/centralised-aadhaar-vault_index', '/content/Index/Home_index', '/content/Index/contact-us_index', '/content/Index/infrastructure_index', '/content/Index/video-conferencing_index', '/content/Index/command-and-control-centre_index', '/content/Index/national-cloud_index', '/content/Index/directory_index', '/content/tabular_data_csv_index', '/content/Text_Index_v1/news_updates', '/content/Text_Index_v1/services', '/content/Text_Index_v1/districts', '/content/Text_Index_v1/events', '/content/Text_Index_v1/photo_gallery', '/content/Text_Index_v1/Nic']

folders = ['/content/Index/Services_index', '/content/Index/rti_index', '/content/Index/webcast_index', '/content/Index/nicnet_index', '/content/Index/messaging_index', '/content/Index/help_index', '/content/Index/Home_index', '/content/Index/infrastructure_index', '/content/Index/website-policies_index', '/content/Index/profile_index', '/content/Index/domain_registration_index', '/content/Index/nkn_index', '/content/Index/web-information-manager_index', '/content/Index/government-local-area-networks-lans_index', '/content/Index/directory_index', '/content/Index/tabular_data_csv_index', '/content/Index/awards_index', '/content/Index/centralised-aadhaar-vault_index', '/content/Index/photo-gallery_index', '/content/Index/contact-us_index', '/content/Index/data-centre_index', '/content/Index/events_index', '/content/Index/remote-sensing-gis_index', '/content/Index/news-update_index', '/content/Index/video-conferencing_index', '/content/Index/security_index', '/content/Index/national-cloud_index', '/content/Index/video-gallery_index', '/content/Index/docs_index', '/content/Index/command-and-control-centre_index']



# Initialize merged data
merged_data = {
    "graph_dict": {}
}

# Loop through each folder
for folder in folders:
    file_path = os.path.join(folder, 'graph_store.json')

    # Load the contents of the JSON file
    with open(file_path, 'r') as file:
        data = json.load(file)

    # Merge the graph_dict
    merged_data['graph_dict']= {**merged_data['graph_dict'],**data['graph_dict']}

# Write the merged data to a new JSON file
with open('/content/combined_index_v4/graph_store.json', 'w') as merged_file:
    json.dump(merged_data, merged_file, indent=4)

In [ ]:
import os
import json

# List of folders
# folders = ['/content/Index/video-gallery_index', '/content/Index/remote-sensing-gis_index', '/content/Index/help_index', '/content/Index/events_index', '/content/Index/Services_index', '/content/Index/domain_registration_index', '/content/Index/data-centre_index', '/content/Index/messaging_index', '/content/Index/web-information-manager_index', '/content/Index/nicnet_index', '/content/Index/website-policies_index', '/content/Index/news-update_index', '/content/Index/government-local-area-networks-lans_index', '/content/Index/profile_index', '/content/Index/webcast_index', '/content/Index/rti_index', '/content/Index/nkn_index', '/content/Index/awards_index', '/content/Index/security_index', '/content/Index/photo-gallery_index', '/content/Index/centralised-aadhaar-vault_index', '/content/Index/Home_index', '/content/Index/contact-us_index', '/content/Index/infrastructure_index', '/content/Index/video-conferencing_index', '/content/Index/command-and-control-centre_index', '/content/Index/national-cloud_index', '/content/Index/directory_index', '/content/tabular_data_csv_index', '/content/text_Index']
# folders = ['/content/Index/video-gallery_index', '/content/Index/remote-sensing-gis_index', '/content/Index/help_index', '/content/Index/events_index', '/content/Index/Services_index', '/content/Index/domain_registration_index', '/content/Index/data-centre_index', '/content/Index/messaging_index', '/content/Index/web-information-manager_index', '/content/Index/nicnet_index', '/content/Index/website-policies_index', '/content/Index/news-update_index', '/content/Index/government-local-area-networks-lans_index', '/content/Index/profile_index', '/content/Index/webcast_index', '/content/Index/rti_index', '/content/Index/nkn_index', '/content/Index/awards_index', '/content/Index/security_index', '/content/Index/photo-gallery_index', '/content/Index/centralised-aadhaar-vault_index', '/content/Index/Home_index', '/content/Index/contact-us_index', '/content/Index/infrastructure_index', '/content/Index/video-conferencing_index', '/content/Index/command-and-control-centre_index', '/content/Index/national-cloud_index', '/content/Index/directory_index', '/content/tabular_data_csv_index', '/content/Text_Index_v1/news_updates', '/content/Text_Index_v1/services', '/content/Text_Index_v1/districts', '/content/Text_Index_v1/events', '/content/Text_Index_v1/photo_gallery', '/content/Text_Index_v1/Nic']

folders = ['/content/Index/Services_index', '/content/Index/rti_index', '/content/Index/webcast_index', '/content/Index/nicnet_index', '/content/Index/messaging_index', '/content/Index/help_index', '/content/Index/Home_index', '/content/Index/infrastructure_index', '/content/Index/website-policies_index', '/content/Index/profile_index', '/content/Index/domain_registration_index', '/content/Index/nkn_index', '/content/Index/web-information-manager_index', '/content/Index/government-local-area-networks-lans_index', '/content/Index/directory_index', '/content/Index/tabular_data_csv_index', '/content/Index/awards_index', '/content/Index/centralised-aadhaar-vault_index', '/content/Index/photo-gallery_index', '/content/Index/contact-us_index', '/content/Index/data-centre_index', '/content/Index/events_index', '/content/Index/remote-sensing-gis_index', '/content/Index/news-update_index', '/content/Index/video-conferencing_index', '/content/Index/security_index', '/content/Index/national-cloud_index', '/content/Index/video-gallery_index', '/content/Index/docs_index', '/content/Index/command-and-control-centre_index']

# Initialize merged data
merged_data = {
    "embedding_dict": {},
    "text_id_to_ref_doc_id": {},
    "metadata_dict": {}
}

# Loop through each folder
for folder in folders:
    file_path = os.path.join(folder, 'image__vector_store.json')

    # Load the contents of the JSON file
    with open(file_path, 'r') as file:
        data = json.load(file)

    # Merge the embedding_dict
    merged_data['embedding_dict'] = {**merged_data['embedding_dict'], **data['embedding_dict']}

    # Merge the text_id_to_ref_doc_id
    merged_data['text_id_to_ref_doc_id'] = {**merged_data['text_id_to_ref_doc_id'], **data['text_id_to_ref_doc_id']}

    # Merge the metadata_dict
    merged_data['metadata_dict'] = {**merged_data['metadata_dict'], **data['metadata_dict']}

# Write the merged data to a new JSON file
with open('/content/combined_index_v4/image_vector_store.json', 'w') as merged_file:
    json.dump(merged_data, merged_file, indent=4)

In [ ]:
# VectorStoreIndex

import os
import json

# List of folders
# folders = ['/content/Index/video-gallery_index', '/content/Index/remote-sensing-gis_index', '/content/Index/help_index', '/content/Index/events_index', '/content/Index/Services_index', '/content/Index/domain_registration_index', '/content/Index/data-centre_index', '/content/Index/messaging_index', '/content/Index/web-information-manager_index', '/content/Index/nicnet_index', '/content/Index/website-policies_index', '/content/Index/news-update_index', '/content/Index/government-local-area-networks-lans_index', '/content/Index/profile_index', '/content/Index/webcast_index', '/content/Index/rti_index', '/content/Index/nkn_index', '/content/Index/awards_index', '/content/Index/security_index', '/content/Index/photo-gallery_index', '/content/Index/centralised-aadhaar-vault_index', '/content/Index/Home_index', '/content/Index/contact-us_index', '/content/Index/infrastructure_index', '/content/Index/video-conferencing_index', '/content/Index/command-and-control-centre_index', '/content/Index/national-cloud_index', '/content/Index/directory_index', '/content/tabular_data_csv_index', '/content/text_Index']
# folders = ['/content/Index/video-gallery_index', '/content/Index/remote-sensing-gis_index', '/content/Index/help_index', '/content/Index/events_index', '/content/Index/Services_index', '/content/Index/domain_registration_index', '/content/Index/data-centre_index', '/content/Index/messaging_index', '/content/Index/web-information-manager_index', '/content/Index/nicnet_index', '/content/Index/website-policies_index', '/content/Index/news-update_index', '/content/Index/government-local-area-networks-lans_index', '/content/Index/profile_index', '/content/Index/webcast_index', '/content/Index/rti_index', '/content/Index/nkn_index', '/content/Index/awards_index', '/content/Index/security_index', '/content/Index/photo-gallery_index', '/content/Index/centralised-aadhaar-vault_index', '/content/Index/Home_index', '/content/Index/contact-us_index', '/content/Index/infrastructure_index', '/content/Index/video-conferencing_index', '/content/Index/command-and-control-centre_index', '/content/Index/national-cloud_index', '/content/Index/directory_index', '/content/tabular_data_csv_index', '/content/Text_Index_v1/news_updates', '/content/Text_Index_v1/services', '/content/Text_Index_v1/districts', '/content/Text_Index_v1/events', '/content/Text_Index_v1/photo_gallery', '/content/Text_Index_v1/Nic']

folders = ['/content/Index/Services_index', '/content/Index/rti_index', '/content/Index/webcast_index', '/content/Index/nicnet_index', '/content/Index/messaging_index', '/content/Index/help_index', '/content/Index/Home_index', '/content/Index/infrastructure_index', '/content/Index/website-policies_index', '/content/Index/profile_index', '/content/Index/domain_registration_index', '/content/Index/nkn_index', '/content/Index/web-information-manager_index', '/content/Index/government-local-area-networks-lans_index', '/content/Index/directory_index', '/content/Index/tabular_data_csv_index', '/content/Index/awards_index', '/content/Index/centralised-aadhaar-vault_index', '/content/Index/photo-gallery_index', '/content/Index/contact-us_index', '/content/Index/data-centre_index', '/content/Index/events_index', '/content/Index/remote-sensing-gis_index', '/content/Index/news-update_index', '/content/Index/video-conferencing_index', '/content/Index/security_index', '/content/Index/national-cloud_index', '/content/Index/video-gallery_index', '/content/Index/docs_index', '/content/Index/command-and-control-centre_index']


# Initialize merged data
merged_data = {
    "index_store/data": {
        "vector_index": {
            "__type__": "vector_store",
            "__data__": "{\"index_id\": \"vector_index\", \"summary\": null, \"nodes_dict\": {}, \"doc_id_dict\": {}, \"embeddings_dict\": {}}"
        }
    }
}


# Loop through each folder
for folder in folders:
    file_path = os.path.join(folder, 'index_store.json')

    # Load the contents of the JSON file
    with open(file_path, 'r') as file:
        data = json.load(file)

    # Merge the vector_index data
    vector_index_data = json.loads(merged_data['index_store/data']['vector_index']['__data__'])
    vector_index_data['nodes_dict'] = {**vector_index_data['nodes_dict'], **json.loads(data['index_store/data']['vector_index']['__data__'])['nodes_dict']}
    vector_index_data['doc_id_dict'] = {**vector_index_data['doc_id_dict'], **json.loads(data['index_store/data']['vector_index']['__data__'])['doc_id_dict']}
    vector_index_data['embeddings_dict'] = {**vector_index_data['embeddings_dict'], **json.loads(data['index_store/data']['vector_index']['__data__'])['embeddings_dict']}
    merged_data['index_store/data']['vector_index']['__data__'] = json.dumps(vector_index_data)

# Write the merged data to a new JSON file
with open('/content/combined_index_v4/index_store.json', 'w') as merged_file:
    json.dump(merged_data, merged_file, indent=4)

# Model and Embeddings

In [49]:
# Initialize settings for LlamaIndex
# Settings.llm = Ollama(model="llama3", temperature=0, request_timeout=500.0)
Settings.llm = Ollama(model="llama3", temperature=0, request_timeout=500.0, max_tokens=80000000000)

In [50]:
Settings.embed_model = HuggingFaceEmbedding(model_name="Alibaba-NLP/gte-large-en-v1.5", trust_remote_code=True)

# Settings.embed_model = HuggingFaceEmbedding(model_name="nvidia/NV-Embed-v1", trust_remote_code=True, use_auth_token=True)


# Loading the index from disk

In [ ]:
persist_dir = "/content/combined_index_v3"

storage_context = StorageContext.from_defaults(persist_dir=persist_dir)
index = load_index_from_storage(storage_context)

print(index)

In [ ]:
!!cat /content/chat_store2.pkl

['"{\\"store\\": {\\"user1\\": [{\\"role\\": \\"user\\", \\"content\\": \\"What can you tell me about the organization structure?\\", \\"additional_kwargs\\": {}}, {\\"role\\": \\"assistant\\", \\"content\\": \\"Based on the provided context, I can tell you that the National Informatics Centre (NIC) Maharashtra State Centre has an organization structure that includes various levels of personnel with different designations and roles. Here\'s a breakdown:\\\\n\\\\n1. **Scientist-F**: This is the highest level in the organization, with 5-6 individuals holding this designation. They report to senior officials.\\\\n2. **Scientist-E**: There are around 15-16 individuals at this level, who report to Scientist-F or Scientist-D.\\\\n3. **Scientist-D**: Approximately 10-11 individuals hold this designation, which is one level below Scientist-E. They report to Scientist-F or Scientist-E.\\\\n4. **Scientist-C**: Around 5-6 individuals are at this level, reporting to Scientist-D or Scientist-E.\\\\

# Queries

In [ ]:
# What are NIC services?
response_json

{'response': 'According to the provided context information, National Informatics Centre (NIC) provides various services including:\n\n1. Electronic Mail (E-Mail) services over NICNET, which is a satellite-based communication network.\n2. SMTP, UUCP, and X.400 email services.\n3. Integration with a X.500 directory for easy searching of e-mail addresses.\n4. Fax messaging through E-Mail.\n5. Messaging services, including core eMail application Gateway services, Short Messaging Service (SMS), Outbound Dialing (OBD), and an IT Platform for citizen engagement (Sampark).\n6. Video Conferencing services for direct interaction with stakeholders.\n7. Webcast services for live/on-demand broadcasts of important events and conferences.\n\nThese services are offered to users across the country, including government departments, ministries, and autonomous bodies.',
 'search_Score': 0.6809562153737795,
 'url': 'https://www.nic.in/servicecontents/messaging/'}

In [ ]:
# What is the published date of Training Program of IVFRT-MMP at FRRO?
response_json

{'response': 'I\'m happy to help! However, I don\'t see any information about a "Training Program of IVFRT-MMP at FRRO" in the provided context. The context only mentions file paths for CSV files and some personal information about Smt. Ireni Akoijam and details about JAWS screen reader. There is no mention of a training program or its published date.',
 'search_Score': 0.47610851359310563,
 'url': 'https://maharashtra.nic.in/rti/'}

In [ ]:
# what is the published date of SAMPARK-NIC eGOV mobile App?
response_json

{'response': 'The published date of SAMPARK – NIC eGov Mobile App is July 31, 2021.',
 'search_Score': 0.683033538785289,
 'url': 'https://maharashtra.nic.in/news-update/'}

In [ ]:
# what is the published date of SIMNIC?
response_json

{'response': 'According to the provided context information, the published date of SIMNIC is August 2, 2021.',
 'search_Score': 0.5385759364469261,
 'url': 'https://maharashtra.nic.in/news-update/'}

In [ ]:
# can you tell me about the latest news?
response_json

{'response': "There is no news update provided in the given context. The context appears to be a list of screen reader information, including the name of the screen reader, its website, and whether it's free or commercial. There is no mention of any news updates.",
 'search_Score': 0.47680748668387585,
 'url': 'https://maharashtra.nic.in/help/'}

In [ ]:
# who is the SIO?
response_json

{'response': 'According to the provided context information, the CISO (Chief Information Security Officer) is responsible for the security of the National Informatics Centre (NIC).',
 'search_Score': 0.6270658935981352,
 'url': 'https://www.nic.in/servicecontents/nkn/'}

In [ ]:
# what are the events?
response_json

{'response': "According to the provided context information, the events that are covered using NIC Webcast services include:\n\n1. Union Budget speech\n2. President's address to the nation\n3. Prime Minister's Mann Ki Baat & other speeches\n4. Independence and Republic Day celebrations at New Delhi\n5. Air Force Day\n6. Dance and cultural Festivals\n7. PIB press conferences\n8. NIC Knowledge sharing\n9. NKN events\n10. Proceedings of state assemblies\n11. National and international events/conferences such as:\n\t* Make in India\n\t* Skill India\n\t* Start-up India\n\t* Digital India\n\t* International Yoga Day",
 'search_Score': 0.4722525002107816,
 'url': 'https://maharashtra.nic.in/infrastructure/'}

In [ ]:
# whom did Ms. D. Lakshmi Prasanna reports to?
response_json

{'response': "Based on the provided context information, I can see that there is no mention of Ms. D. Lakshmi Prasanna's name or designation in the list of scientists and officials. Therefore, it is not possible to determine who her reporting officer would be based on this information alone.",
 'search_Score': 0.7028144708896884,
 'url': 'https://maharashtra.nic.in/directory/'}

In [ ]:
# what is the organization structure?
response_json

{'response': 'The organization structure appears to be a hierarchical structure with various levels of officials and staff. The top-level officials include Scientists-F, who report to higher-level officials such as Scientist-D or Scientist-C. There are also lower-level officials like Section Officers, Senior Secretariat Assistants, Junior Secretariat Assistants, and Staff Car Drivers. Additionally, there are Multitasking Staff and Steno Gr-II personnel.',
 'search_Score': 0.47507247362784927,
 'url': 'https://maharashtra.nic.in/organization-structure/'}

In [ ]:
# who is the reporting officer of Ms. D. Lakshmi Prasanna?
response_json

{'response': 'There is no mention of a photo gallery in the provided context information. The context appears to be related to screen readers and document types, but it does not include any information about a photo gallery. Therefore, I cannot provide an answer to this query based on the given context.',
 'search_Score': 0.5215259570020727,
 'url': 'https://maharashtra.nic.in/help/'}

In [ ]:
# what is there in the video gallery?
response_json

{'response': 'According to the provided context, the Video Gallery contains videos related to NIC Maharashtra – ICT support during COVID-19.',
 'search_Score': 0.5873177463100648,
 'url': 'https://maharashtra.nic.in/video-gallery/'}

In [ ]:
response_json

{'response': 'According to the provided context information, National Informatics Centre (NIC) has been involved in various works, including:\n\n1. Establishing High Capacity SCPC VSAT Connectivity at Kavarati, Lakshadweep and Port Blair, Andaman & Nicobar Island.\n2. Providing Data Centers Services from National Data Centres at Delhi, Hyderabad, Pune, and Bhubaneswar.\n3. Upgrading the National Data Centre at Delhi with high-speed network backbone, 1.6 Petabyte Enterprise-class storage, high-throughput Network Load Balancers, and Intrusion Prevention Systems.\n4. Implementing Solution for Backup as a Service & Storage as a Service.\n5. Hosting/enhancing ICT infrastructure of national-level projects, including E-office, e-Courts, and e-Transport.\n6. Upgrading Data Centres at Pune and Hyderabad with high-speed network backbone and storage capacity enhanced.\n7. Renovating the National Data Centre at Hyderabad with a capacity of 106 Racks.\n8. Launching National Cloud Services under Meg

# Visualizing Reranking

In [ ]:
persist_dir = "/content/combined_index"

storage_context = StorageContext.from_defaults(persist_dir=persist_dir)
index = load_index_from_storage(storage_context)

print(index)

In [ ]:
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core import QueryBundle
import pandas as pd
from IPython.display import display, HTML
from copy import deepcopy

pd.set_option("display.max_colwidth", None)

# configure retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=10)

def get_retrieved_nodes(
    query_str, reranker
):
    query_bundle = QueryBundle(query_str)

    retrieved_nodes = retriever.retrieve(query_bundle)

    if reranker != "None":
      retrieved_nodes = reranker.postprocess_nodes(retrieved_nodes, query_bundle)
    else:
        retrieved_nodes

    return retrieved_nodes


def pretty_print(df):
    return display(HTML(df.to_html().replace("\\n", "<br>")))


def visualize_retrieved_nodes(nodes) -> None:
    result_dicts = []
    for node in nodes:
        node = deepcopy(node)
        node.node.metadata = {}
        node_text = node.node.get_text()
        node_text = node_text.replace("\n", " ")

        result_dict = {"Score": node.score, "Text": node_text}
        result_dicts.append(result_dict)

    pretty_print(pd.DataFrame(result_dicts))

# Comparing with reranker and without reranker

In [ ]:
# Define all embeddings and rerankers

RERANKERS = {
    "WithoutReranker": "None",
    "cross-encoder": SentenceTransformerRerank(model="cross-encoder/ms-marco-MiniLM-L-2-v2", top_n=5),
    "bge-reranker-large": SentenceTransformerRerank(model="BAAI/bge-reranker-large", top_n=5)
}

NameError: name 'SentenceTransformerRerank' is not defined

In [ ]:
RERANKERS.items()

dict_items([('WithoutReranker', 'None'), ('cross-encoder', SentenceTransformerRerank(callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7ca0e5a8f340>, model='cross-encoder/ms-marco-MiniLM-L-2-v2', top_n=5, device='cuda', keep_retrieval_score=False)), ('bge-reranker-large', SentenceTransformerRerank(callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7c9fec1c8be0>, model='BAAI/bge-reranker-large', top_n=5, device='cuda', keep_retrieval_score=False))])

In [ ]:
query_str = "what are the works done by NIC?"

# Loop over rerankers
for rerank_name, reranker in RERANKERS.items():
    print(f"Running Evaluation for Reranker: {rerank_name}")

    query_bundle = QueryBundle(query_str)

    retrieved_nodes = retriever.retrieve(query_bundle)

    if reranker != "None":
      retrieved_nodes = reranker.postprocess_nodes(retrieved_nodes, query_bundle)
    else:
        retrieved_nodes

    print(f"Visualize Retrieved Nodes for Reranker: {rerank_name}")
    visualize_retrieved_nodes(retrieved_nodes)

Running Evaluation for Reranker: WithoutReranker
Visualize Retrieved Nodes for Reranker: WithoutReranker


Running Evaluation for Reranker: cross-encoder
Visualize Retrieved Nodes for Reranker: cross-encoder


Running Evaluation for Reranker: bge-reranker-large
Visualize Retrieved Nodes for Reranker: bge-reranker-large


# Testing Query_engine with Rerank

In [51]:
file_to_url_mapping = {'news_updates.docx': 'https://maharashtra.nic.in/news-update/',
'Nic.docx': 'https://maharashtra.nic.in/',
'districts.docx': 'https://maharashtra.nic.in/district-centres/',
'events.docx': 'https://maharashtra.nic.in/events/',
'services.docx': 'https://maharashtra.nic.in/services/',
'photo_gallery.docx': 'https://maharashtra.nic.in/photo-gallery/',
'Screen Reader Information.csv': 'https://maharashtra.nic.in/help/',
 'Plug-in for alternate document types.csv': 'https://maharashtra.nic.in/help/',
'SIO (State Informatics Officer).csv': 'https://maharashtra.nic.in/directory/',
 'Maharashtra State Centre, Mumbai.csv': 'https://maharashtra.nic.in/directory/',
 'Ahmadnagar.csv': 'https://maharashtra.nic.in/directory/',
 'Akola.csv': 'https://maharashtra.nic.in/directory/',
 'Amravati.csv': 'https://maharashtra.nic.in/directory/',
 'Aurangabad.csv': 'https://maharashtra.nic.in/directory/',
 'Beed.csv': 'https://maharashtra.nic.in/directory/',
 'Bhandara.csv': 'https://maharashtra.nic.in/directory/',
 'Buldana.csv': 'https://maharashtra.nic.in/directory/',
 'Chandrapur.csv': 'https://maharashtra.nic.in/directory/',
 'Dhule.csv': 'https://maharashtra.nic.in/directory/',
'District Centres (final test).csv': 'https://maharashtra.nic.in/district-centres/',
 'Gadchiroli.csv': 'https://maharashtra.nic.in/directory/',
 'Gondia.csv': 'https://maharashtra.nic.in/directory/',
 'Hingoli.csv': 'https://maharashtra.nic.in/directory/',
 'Jalgaon.csv': 'https://maharashtra.nic.in/directory/',
 'Jalna.csv': 'https://maharashtra.nic.in/directory/',
 'Kolhapur.csv': 'https://maharashtra.nic.in/directory/',
 'Mumbai.csv': 'https://maharashtra.nic.in/directory/',
 'Mumbai Suburban (Bandra).csv': 'https://maharashtra.nic.in/directory/',
 'Nagpur.csv': 'https://maharashtra.nic.in/directory/',
 'Nanded.csv': 'https://maharashtra.nic.in/directory/',
 'Nandurbar.csv': 'https://maharashtra.nic.in/directory/',
 'Nasik.csv': 'https://maharashtra.nic.in/directory/',
 'Osmanabad.csv': 'https://maharashtra.nic.in/directory/',
 'Palghar.csv': 'https://maharashtra.nic.in/directory/',
 'Parbhani.csv': 'https://maharashtra.nic.in/directory/',
 'Pune District.csv': 'https://maharashtra.nic.in/directory/',
'Reporting_officer.csv': 'https://maharashtra.nic.in/organization-structure/',
 'Raigad.csv': 'https://maharashtra.nic.in/directory/',
 'Ratnagiri.csv': 'https://maharashtra.nic.in/directory/',
 'Sangli.csv': 'https://maharashtra.nic.in/directory/',
 'Satara.csv': 'https://maharashtra.nic.in/directory/',
 'Sindhu Durg.csv': 'https://maharashtra.nic.in/directory/',
 'Solapur.csv': 'https://maharashtra.nic.in/directory/',
 'Thane.csv': 'https://maharashtra.nic.in/directory/',
 'Wardha.csv': 'https://maharashtra.nic.in/directory/',
 'Washim.csv': 'https://maharashtra.nic.in/directory/',
 'Yavatmal.csv': 'https://maharashtra.nic.in/directory/',
 'APPELLATE AUTHORITY.csv': 'https://maharashtra.nic.in/rti/',
 'PUBLIC INFORMATION OFFICERS ( PIO ).csv': 'https://maharashtra.nic.in/rti/'}

# Post Processors


In [28]:
from llama_index.core.postprocessor import LongContextReorder, SentenceEmbeddingOptimizer
from llama_index.core.postprocessor import (
    FixedRecencyPostprocessor,
    EmbeddingRecencyPostprocessor,
)

persist_dir = "/content/webpages_doc_index"
storage_context = StorageContext.from_defaults(persist_dir=persist_dir)
index = load_index_from_storage(storage_context)

print(index)

reorder = LongContextReorder()
# node_postprocessor_emb = EmbeddingRecencyPostprocessor()

reorder_engine = index.as_chat_engine(
    node_postprocessors=[reorder], similarity_top_k=10
)

# reorder_engine = index.as_chat_engine(
#     node_postprocessors=[SentenceEmbeddingOptimizer(percentile_cutoff=0.7)]
# )

base_engine = index.as_chat_engine(similarity_top_k=10)

In [ ]:
from llama_index.core.response.notebook_utils import display_response

base_response = base_engine.chat("What is published date of the Training Program of IVFRT-MMP at FRRO?")
display_response(base_response)

**`Final Response:`** The published date of the Training Program of IVFRT-MMP at FRRO is August 20, 2013.

In [ ]:
base_response

AgentChatResponse(response='The published date of the Training Program of IVFRT-MMP at FRRO is August 20, 2013.', sources=[ToolOutput(content='According to the provided context information, the published date of the "Training Program of IVFRT-MMP at FRRO" is August 20, 2013.', tool_name='query_engine_tool', raw_input={'input': 'published date of the Training Program of IVFRT-MMP at FRRO'}, raw_output=Response(response='According to the provided context information, the published date of the "Training Program of IVFRT-MMP at FRRO" is August 20, 2013.', source_nodes=[NodeWithScore(node=TextNode(id_='90290cbd-19a7-4594-9076-f07b68779323', embedding=None, metadata={'file_name': 'news_updates.docx', 'file_path': '/content/docs/news_updates.docx', 'file_type': 'application/vnd.openxmlformats-officedocument.wordprocessingml.document', 'file_size': 15496, 'creation_date': '2024-06-06', 'last_modified_date': '2024-06-06'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'cr

In [ ]:
node = base_response.source_nodes[0]
response_json = {}
response_json['response'] = base_response.response
response_json['Search_Score'] = node.score

try:
  response_json['url'] = str(node.metadata['URL'])
except  KeyError as e:
  response_json['url'] =  str(file_to_url_mapping[node.metadata['file_name']])

print(response_json)

{'response': 'The published date of the Training Program of IVFRT-MMP at FRRO is August 20, 2013.', 'Search_Score': 0.4379604420790352, 'url': 'https://maharashtra.nic.in/news-update/'}


In [ ]:
print(base_response.get_formatted_sources())

> Source (Doc id: 1f49a56e-9966-4bb0-a2c7-e5b1c387e46a): The following are the list of photo titles which are available in the photo gallery section of th...

> Source (Doc id: 90290cbd-19a7-4594-9076-f07b68779323): Following are the news updates available in the official website of NIC Maharashtra https://mahar...

> Source (Doc id: dce2aafa-31bf-494f-98e3-454961649537): Below is the list of all the 5 events that took place in NIC as mentioned in the official website...

> Source (Doc id: 2f4b8ee2-5eb0-449c-8204-9746ac19217d): 1. What is NIC? or Introduction of NIC or Profile of NIC

Answer:

National Informatics Centre (N...

> Source (Doc id: f897f45e-34aa-475f-a8f2-3310a9b51765): Here is the list of 35 districts/district centres where National Informatics Centre (NIC) has its...

> Source (Doc id: a659c0ac-15b7-4cee-b8b5-9fc68d21dd65): Overview of Services offered by NIC:

NIC is closely associated with the government in different ...


In [29]:
reorder_response = reorder_engine.chat("What is published date of the eFile implementation at NRHM-Mumbai?")
display_response(reorder_response)

ValueError: Reached max iterations.

In [ ]:
node = reorder_response.source_nodes[0]
response_json = {}
response_json['response'] = reorder_response.response
response_json['Search_Score'] = node.score

try:
  response_json['url'] = str(node.metadata['URL'])
except  KeyError as e:
  response_json['url'] =  str(file_to_url_mapping[node.metadata['file_name']])

print(response_json)

{'response': 'The published date of the eFile implementation at NRHM-Mumbai is March 13, 2012.', 'Search_Score': 0.7125760023009837, 'url': 'https://maharashtra.nic.in/events/'}


In [ ]:
print(reorder_response.get_formatted_sources())

> Source (Doc id: 90290cbd-19a7-4594-9076-f07b68779323): Following are the news updates available in the official website of NIC Maharashtra https://mahar...

> Source (Doc id: 2f4b8ee2-5eb0-449c-8204-9746ac19217d): 1. What is NIC? or Introduction of NIC or Profile of NIC

Answer:

National Informatics Centre (N...

> Source (Doc id: a659c0ac-15b7-4cee-b8b5-9fc68d21dd65): Overview of Services offered by NIC:

NIC is closely associated with the government in different ...

> Source (Doc id: f897f45e-34aa-475f-a8f2-3310a9b51765): Here is the list of 35 districts/district centres where National Informatics Centre (NIC) has its...

> Source (Doc id: dce2aafa-31bf-494f-98e3-454961649537): Below is the list of all the 5 events that took place in NIC as mentioned in the official website...

> Source (Doc id: 1f49a56e-9966-4bb0-a2c7-e5b1c387e46a): The following are the list of photo titles which are available in the photo gallery section of th...


In [ ]:
reorder_response

AgentChatResponse(response='The published date of the Training Program of IVFRT-MMP at FRRO is August 20, 2013.', sources=[ToolOutput(content='According to the provided context information, the published date of the Training Program of IVFRT-MMP at FRRO is August 20, 2013.', tool_name='query_engine_tool', raw_input={'input': 'published date of the Training Program of IVFRT-MMP at FRRO'}, raw_output=Response(response='According to the provided context information, the published date of the Training Program of IVFRT-MMP at FRRO is August 20, 2013.', source_nodes=[NodeWithScore(node=TextNode(id_='1f49a56e-9966-4bb0-a2c7-e5b1c387e46a', embedding=None, metadata={'file_name': 'photo_gallery.docx', 'file_path': '/content/docs/photo_gallery.docx', 'file_type': 'application/vnd.openxmlformats-officedocument.wordprocessingml.document', 'file_size': 14816, 'creation_date': '2024-06-06', 'last_modified_date': '2024-06-06'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'crea

# Print the response

In [52]:
from llama_index.core.storage.chat_store import SimpleChatStore
from llama_index.core.memory import ChatMemoryBuffer
from llama_index.core import VectorStoreIndex, get_response_synthesizer
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.postprocessor import LongContextReorder
from llama_index.core.postprocessor import SentenceTransformerRerank


persist_dir = "/content/combined_data_index"
storage_context = StorageContext.from_defaults(persist_dir=persist_dir)
index = load_index_from_storage(storage_context)

print(index)

# reorder = LongContextReorder()
rerank = SentenceTransformerRerank(model="BAAI/bge-reranker-large", top_n=7)


chat_store = SimpleChatStore()

chat_memory = ChatMemoryBuffer.from_defaults(
    token_limit=30000000,
    chat_store=chat_store,
    chat_store_key="user1",

)

chat_engine = index.as_chat_engine(
    chat_mode="context",
    system_prompt="You are a helpful AI assistant. You are expert in retrieving the answer for the user input based on the provided context. Important note: You must not mention any file name or file path in your response.",
    memory=chat_memory,
    similarity_top_k=10,
    node_postprocessors=[rerank],
    verbose = True
)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
response = query_engine.query("What is published date of AGMARKNET Workshops in Maharashtra by NIC Mumbai?")

node = response.source_nodes[0]
response_json = {}
response_json['response'] = response.response
response_json['Search_Score'] = node.score

try:
  response_json['url'] = str(node.metadata['URL'])
except  KeyError as e:
  response_json['url'] =  str(file_to_url_mapping[node.metadata['file_name']])

print(response_json)

{'response': 'I\'m unable to find any information about a "Training Program of IVFRT-MMP at FRRO" in the provided context. The files and data mentioned appear to be related to personnel information, district centers, and screen readers, but there is no mention of a training program or its published date.', 'Search_Score': 0.4713932929016982, 'url': 'https://maharashtra.nic.in/rti/'}


In [56]:
response = chat_engine.chat("to whom did Ms. Meera Joshi reports?")

node = response.source_nodes[0]
response_json = {}
response_json['response'] = response.response
response_json['Search_Score'] = node.score

try:
  response_json['url'] = str(node.metadata['URL'])
except  KeyError as e:
  response_json['url'] =  str(file_to_url_mapping[node.metadata['file_name']])

print(response_json)

{'response': 'According to the provided context, Ms. Meera Joshi reports to Shri Manoj Kumar Mishra.', 'Search_Score': 0.91262907, 'url': 'https://maharashtra.nic.in/organization-structure/'}


In [ ]:
response_json

{'response': 'The published date of the "Training Program of IVFRT-MMP at FRRO" is August 20, 2013.',
 'Search_Score': 0.00040770933,
 'url': 'https://maharashtra.nic.in/news-update/'}

In [ ]:
response_json

{'response': 'The published date of AGMARKNET Workshops in Maharashtra by NIC Mumbai is December 29, 2004.',
 'Search_Score': 9.792949e-05,
 'url': 'https://maharashtra.nic.in/events/'}

In [ ]:
response

AgentChatResponse(response='The published date of AGMARKNET Workshops in Maharashtra by NIC Mumbai is December 29, 2004.', sources=[ToolOutput(content='system: You are a helpful AI assistant. You are expert in retrieving the answer for the user input based on the provided context. Important note: You must not mention any file name or file path in your response.\nContext information is below.\n--------------------\nfile_path: /content/docs/events.docx\n\nBelow is the list of all the 5 events that took place in NIC as mentioned in the official website of NIC maharashtra:\n\nCyber Crime and Cyber Laws (Start Date: 28/09/2022, End Date: 28/09/2022, Venue: NIC Maharashtra State Centre, Mumbai) Webinar on “Cyber Crime and Cyber Laws ” at NIC Maharashtra State Centre, Mumbai September 28th, 2022.\n\nSuperannuation of Sh. Dhanjay Kulkarni, DIO Pune and Ms. Sneha Shula, ADIO Nashik in the Month of April 2022, (Start Date: 30/04/2022, End Date : 30/04/2022, Venue : Mumbai)\n\nSIO Maharashtra met

In [ ]:
response_json

{'response': 'The published date of SAMPARK is July 31, 2021.',
 'Search_Score': 0.81622535,
 'url': 'https://maharashtra.nic.in/news-update/'}

In [ ]:
response_json

{'response': 'The eGov Maharashtra was held at Mumbai on May 9-10, 2013.',
 'Search_Score': 0.007553987,
 'url': 'https://maharashtra.nic.in/news-update/'}

In [ ]:
# tell me about organization structure
response_json

{'response': "Based on the provided context, I can infer that the organization is likely a government agency or department in India. Here's an attempt to outline the organization structure:\n\n1. **State Informatics Officer (SIO)**: This appears to be a senior-level position responsible for overseeing informatics activities at the state level.\n2. **Scientist-F**: This designation seems to refer to a technical expert or specialist, possibly with a focus on information technology or computer science.\n3. **Junior Secretariat Assistant**: This is likely an entry-level administrative support role.\n4. **Reporting Officer**: This term suggests that there may be a hierarchical structure within the organization, where certain individuals report to specific officers or supervisors.\n\nThe organization seems to have a mix of technical and administrative roles, with a focus on information technology and computer science. The presence of a State Informatics Officer and Public Information Officer

In [ ]:
# Who is the SIo
response_json

{'response': 'According to the provided context, the SIO (State Informatics Officer) is Ms. Sapna Kapoor, who holds the designation of Scientist-F and has an email address of sapna[dot]kapoor[at]nic[dot]in.',
 'Search_Score': 0.046427052,
 'url': 'https://maharashtra.nic.in/directory/'}

In [ ]:
# How many districts centres are there?
response_json

{'response': 'There are 35 district centers where National Informatics Centre (NIC) has its presence.',
 'Search_Score': 0.6682483,
 'url': 'https://maharashtra.nic.in/district-centres/'}

In [ ]:
# What is the published date of SIMNIC?
response_json

{'response': 'The published date of SIMNIC is August 2, 2021.',
 'Search_Score': 0.96835625,
 'url': 'https://maharashtra.nic.in/news-update/'}

In [ ]:
# What are NIC services?
response_json

{'response': 'According to the provided context, NIC (National Informatics Centre) provides various services including:\n\n1. National Cloud\n2. Messaging\n3. Video Conferencing\n4. Domain Registration and Webcast\n5. Multi-Gigabit Nationwide Networks (NICNET)\n6. National Data Centres\n7. National Cloud\n8. Pan-India VC Infrastructure\n9. Command and Control Centre\n10. Multi-Layered GIS-Based Platform\n\nThese services are designed to support e-governance initiatives, provide infrastructure for government departments, and facilitate communication and collaboration among stakeholders.',
 'Search_Score': 0.9801593,
 'url': 'https://maharashtra.nic.in/services/'}

In [ ]:
# What is the published date of Training Program of IVFRT-MMP at FRRO?
response_json

{'response': 'I\'m happy to help! However, I don\'t see any information about a news article titled "Training Program of IVFRT-MMP at FRRO, Mumbai" in the provided context. The context appears to be a list of personnel with their names, designations, emails, phones, and IP numbers. There is no mention of a news article or its published date. If you could provide more information or clarify what you\'re looking for, I\'d be happy to try and assist you further!',
 'Search_Score': 9.042643e-05,
 'url': 'https://maharashtra.nic.in/directory/'}

In [ ]:
#  What is the published date of Implementation of Aadhaar Enabled Biometric Attendance System (AEBAS) in NIC?
response_json

{'response': 'According to the provided context, the published date of Implementation of Aadhaar Enabled Biometric Attendance System (AEBAS) in NIC is November 21, 2014.',
 'Search_Score': 0.9996412,
 'url': 'https://maharashtra.nic.in/news-update/'}

In [ ]:
# What is the published date of Inauguration of Direct Benefit Transfer (AADHAAR) in pilot District Wardha (Maharashtra)?
response_json

{'response': 'According to the provided context, the published date of Inauguration of Direct Benefit Transfer (AADHAAR) in pilot District Wardha (Maharashtra) is January 1, 2013.',
 'Search_Score': 0.0042579635,
 'url': 'https://maharashtra.nic.in/news-update/'}

In [ ]:
# When was the Training Program of IVFRT-MMP at FRRO, Mumbai took place?
response_json

{'response': 'According to the provided context information, the published date of the news article titled "Training Program of IVFRT-MMP at FRRO, Mumbai" is August 20, 2013.',
 'Search_Score': 0.0035063014,
 'url': 'https://maharashtra.nic.in/news-update/'}

In [ ]:
# what is the published date of SIMNIC?
response_json

{'response': 'August 2, 2021',
 'Search_Score': 0.96835625,
 'url': 'https://maharashtra.nic.in/news-update/'}

In [ ]:
# what is the published date of SAMPARK?
response_json

{'response': 'According to the provided context, SAMPARK is a NIC eGov Mobile App that provides information about Government officials (in service) about their present posting details and keeps it updated. The main aim of this application is to provide information to Government officials about their current postings.',
 'Search_Score': 0.57889265,
 'url': 'https://maharashtra.nic.in/news-update/'}

In [ ]:
# What is the news update?
response_json

{'response': 'There is no news update. The provided context appears to be a list of screen reader information, including their names, websites, and whether they are free or commercial. There is no indication of any news-related updates.',
 'Search_Score': 7.6261866e-05,
 'url': 'https://maharashtra.nic.in/help/'}

In [ ]:
# Who is the SIO?
response_json

{'response': 'Based on the provided context information, there is no mention of a person named "SIO". However, we do have two files with paths `/content/tabular_data_csv/SIO.csv` and `/content/tabular_data_csv/APPELLATE AUTHORITY.csv`. Since the query asks about the SIO, I would say that the SIO refers to the file path `/content/tabular_data_csv/SIO.csv`, but there is no information provided about who this person might be.',
 'Search_Score': 0.0002616312,
 'url': 'https://maharashtra.nic.in/directory/'}

In [ ]:
# What are the events?
response_json

{'response': "Based on the provided context, the events mentioned are:\n\n1. Cyber Crime and Cyber Laws (September 28th, 2022)\n2. Superannuation of Sh. Dhanjay Kulkarni and Ms. Sneha Shula (April 30th, 2022)\n3. SIO Maharashtra met new Director General, NIC (June 1st, 2022)\n4. Hon'ble Minister of Electronics and Information Technology and Railways visit to Mumbai (February 18th, 2022)\n5. Swachhatta Pakhwada from 01st to 15th February 2022\n\nThese events are mentioned in the file_path: /content/text/events.txt context.",
 'Search_Score': 0.016033381,
 'url': 'https://maharashtra.nic.in/photo-gallery/'}

In [ ]:
# what is the organization structure?
response_json

{'response': 'The organization structure consists of various officials with their respective designations, reporting to other officials. The officials include:\n\n1. Ms. D. Lakshmi Prasanna - Scientist-F\nReporting To: Shri Anand Swarup Srivastava\n2. Shri C.Prasanna - Scientist-E\nReporting To: Ms. D. Lakshmi Prasanna\n3. Shri Syed Ajaz Gulab - Scientist-E\nReporting To: Ms. D. Lakshmi Prasanna\n4. Shri Rajendra F. Hatwar - Scientist-D\nReporting To: Ms. D. Lakshmi Prasanna\n5. Shri Kunal Vijay Bhamare - Scientist-D\nReporting To: Shri Rajendra F. Hatwar\n6. Shri Saravana Kumar - Scientist-B\nReporting To: Shri Kunal Vijay Bhamare\n\nAnd so on, with multiple officials listed under various categories such as Scientist-F, Scientist-E, Scientist-D, and more.',
 'Search_Score': 0.7342759,
 'url': 'https://maharashtra.nic.in/organization-structure/'}

In [ ]:
# whom did Ms. D. Lakshmi Prasanna reports to?
response_json

{'response': 'Shri Sunil Digambarro Rao Potekar.',
 'Search_Score': 0.02319305,
 'url': 'https://maharashtra.nic.in/directory/'}

In [ ]:
# what is there in the video gallery?
response_json

{'response': "According to the provided context information, the Video Gallery contains a collection of videos showcasing NIC Maharashtra's ICT support during COVID-19.",
 'Search_Score': 0.024777586,
 'url': 'https://maharashtra.nic.in/video-gallery/'}

In [ ]:
response_json

{'response': "Based on the provided context information, National Informatics Centre (NIC) provides various services and platforms to facilitate e-Governance applications in Government Ministries/Departments at the Centre, States, Districts, and Block level. Some of the notable works done by NIC include:\n\n1. Providing different kinds of E-mail services over NICNET, which is NIC's satellite-based communication network.\n2. Establishing a robust Messaging framework that includes core eMail application Gateway services, Short Messaging Service (SMS), OBD (Outbound Dialing), and an IT Platform for citizen engagement (Sampark).\n3. Developing and hosting various websites and portals, such as the NIC website, which provides information on NIC's services, products, and platforms.\n4. Providing institutional linkages across all the Ministries /Departments of the Central Government, State Governments, Union Territories, and District administrations of the country through its Information and C

# Chat_Engine and Storing

In [ ]:
from llama_index.core.storage.chat_store import SimpleChatStore
from llama_index.core.memory import ChatMemoryBuffer

chat_store = SimpleChatStore()

chat_memory1 = ChatMemoryBuffer.from_defaults(
    token_limit=30000000,
    chat_store=chat_store,
    chat_store_key="user1",

)

chat_engine = index.as_chat_engine(
    chat_mode="context",
    system_prompt="You are a helpful AI assistant. You are expert in retrieving the answer for the user input based on the provided context. Important note: You must not mention any file name or file path in your response.",
    memory=chat_memory1, verbose = True, similarity_top_k=10, node_postprocessors=[rerank]
)



In [ ]:
val_query_response = chat_engine.chat("Can you please keep in mind the following information: Training Program of IVFRT-MMP at FRRO, Mumbai on 20th August 2013, C-FRO200813, Publish Date : August 20, 2013. Also direct the user to link/url: https://maharashtra.nic.in/news-update/  .")

In [ ]:
val_query_response = chat_engine.chat("Can you tell me the start and end date of Swachhatta Pakhwada event?")

In [ ]:
val_query_response

AgentChatResponse(response="Based on the provided context, National Informatics Centre (NIC) has been involved in various activities and projects across different domains. Here's a summary of some of the key works done by NIC:\n\n**Digital Services:**\n\n* Cloud services\n* Domain registration\n* Email services\n* Security solutions\n* Hosting services\n* Video-conferencing\n\n**Infrastructure:**\n\n* Data centers\n* Networking services\n* Office-automation solutions\n\n**ICT Support:**\n\n* Projects like biometric attendance, e-Office, messaging, cyber security, and webcasting\n* Development of digital agriculture platforms, fintech, health-tech, and edu-tech solutions\n\n**Development Projects:**\n\n* Digital agriculture platforms\n* Fintech solutions\n* Health-tech solutions\n* Edu-tech solutions\n\n**GIS-based Initiatives:**\n\n* GIS portal for Slum Rehabilitation Schemes (SRA) in Mumbai\n* Mobile app for SRA, providing citizen-centric information on slum rehabilitation schemes\n\n

In [ ]:
val_query_response.response

'I apologize for missing that earlier! Thank you for providing the additional context. According to the provided information, the published date of the news article titled "Training Program of IVFRT-MMP at FRRO, Mumbai" is August 20, 2013.'

In [ ]:
node = val_query_response.source_nodes[0]
response_json = {}
response_json['response'] = val_query_response.response
response_json['search_Score'] = node.score

try:
  response_json['url'] = str(node.metadata['URL'])
except  KeyError as e:
  response_json['url'] =  str(file_to_url_mapping[node.metadata['file_name']])




In [ ]:
response_json

{'response': 'I\'ve taken note of the additional context! Thank you for providing the specific information about the news article titled "Training Program of IVFRT-MMP at FRRO, Mumbai". I\'ll make sure to keep this in mind as we continue our conversation.\n\nTo answer your original question: The published date of the news article titled "Training Program of IVFRT-MMP at FRRO, Mumbai" is August 20, 2013.',
 'search_Score': 0.030322898,
 'url': 'https://maharashtra.nic.in/photo-gallery/'}

In [54]:
chat_store.persist(persist_path="/content/chat_store_v3.pkl")


In [55]:
chat_store

SimpleChatStore(store={'user1': [ChatMessage(role=<MessageRole.USER: 'user'>, content='Please remember that the Training Program of IVFRT-MMP at FRRO, Mumbai on 20th August 2013, Publish Date : August 20, 2013. this info is from the url https://maharashtra.nic.in/news-update/   ', additional_kwargs={}), ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content="I apologize for not mentioning any file name or file path in my previous response. Here's a helpful AI assistant response:\n\nThe Training Program of IVFRT-MMP at FRRO, Mumbai on 20th August 2013, Publish Date : August 20, 2013 is an important news update from the official website of NIC Maharashtra https://maharashtra.nic.in/news-update/. This event highlights the importance of training and capacity building in the field of IVFRT-MMP.", additional_kwargs={})]})

# Loading the memory


In [ ]:
persist_dir = "/content/combined_index_v3"

storage_context = StorageContext.from_defaults(persist_dir=persist_dir)
index1 = load_index_from_storage(storage_context)

print(index)

In [ ]:
# prompt: Write the code to retrieve chats from chat_store_old.pkl file and then query the chat_engine
from llama_index.core.storage.chat_store import SimpleChatStore
from llama_index.core.memory import ChatMemoryBuffer

# # Load the chat store from the file
# loaded_chat_store1 = SimpleChatStore.from_persist_path(
#     persist_path="/content/combined_index_v1.zip"
# )

store = SimpleChatStore()

chat_memory = ChatMemoryBuffer.from_defaults(
    token_limit=300000,
    chat_store=store,
    chat_store_key="user1",

)


# Initialize the chat engine
chat_engine1 = index1.as_chat_engine(
    chat_mode="context",
    system_prompt="You are a helpful AI assistant. You are expert in retrieving the answer for the user input based on the provided context. You must not mention any file name or file path in your response.",
    memory=chat_memory, verbose = True, similarity_top_k=10, node_postprocessors=[rerank]
)



In [ ]:
response = chat_engine1.chat("what is the publish date of news article titled as Training Program of IVFRT-MMP at FRRO?")


In [ ]:
node = response.source_nodes[0]
response_json = {}
response_json['response'] = response.response
response_json['search_Score'] = node.score

try:
  response_json['url'] = str(node.metadata['URL'])
except  KeyError as e:
  response_json['url'] =  str(file_to_url_mapping[node.metadata['file_name']])


In [ ]:
# Can you tell me about  Swachhatta Pakhwada ?
response_json

{'response': 'According to the context, Swachhatta Pakhwada was observed at NIC Maharashtra State and District Centres from 1st to 15th February 2022. During this period, all officers and staff from State centre and District centres took the Swachhatta Pledge on 2nd Feb. 2022.\n\nAdditionally, with reference to the circular No. A-60019/04/2021/GCS-1 dated 28/01/2022 from NIC HQ regarding disposal of obsolete Computer Hardware and its related peripherals and furniture items during the Swachhatta Pakhwada 2022, NIC Maharashtra has auctioned obsolete hardware, old books, and weeded files as per the guidelines. Also, initiated the process of auctioning obsolete air conditioners, old furniture, and remaining obsolete hardware from State and District Centres of Maharashtra.',
 'search_Score': 0.13719939,
 'url': 'https://maharashtra.nic.in/events/'}

In [ ]:
# What are the events that took place?
response_json

{'response': 'Based on the provided context, the following events took place at NIC:\n\n1. Cyber Crime and Cyber Laws Webinar (Start Date: 28/09/2022, End Date: 28/09/2022, Venue: NIC Maharashtra State Centre, Mumbai)\n2. Superannuation of Sh. Dhanjay Kulkarni, DIO Pune and Ms. Sneha Shula, ADIO Nashik in the Month of April 2022 (Start Date: 30/04/2022, End Date: 30/04/2022, Venue: Mumbai)\n3. SIO Maharashtra met new Director General, NIC on 01/06/2022 (Start Date: 01/06/2022, End Date: 01/06/2022, Venue: New Delhi)\n4. Hon’ble Minister of Electronics and Information Technology and Railways visit to Mumbai (Start Date: 18/02/2022, End Date: 18/02/2022, Venue: Mumbai)\n5. Swachhatta Pakhwada from 01st to 15th February 2022 (Start Date: 01/02/2022, End Date: 15/02/2022, Venue: NIC Maharashtra State and District Centres)\n\nPlease note that these events are specific to the context provided and may not be an exhaustive list of all events that took place at NIC.',
 'search_Score': 0.0470416

In [ ]:
response_json

{'response': 'I\'m happy to help! However, I don\'t see any information about a news article titled "Training Program of IVFRT-MMP at FRRO" in the provided context. The context appears to be related to personnel data and does not mention any specific news articles or publish dates. If you could provide more information or clarify which training program you are referring to, I\'ll do my best to assist you.',
 'search_Score': 7.7409335e-05,
 'url': 'https://maharashtra.nic.in/directory/'}

In [ ]:
# Query the chat engine
response = chat_engine1.chat("Can you tell me about Swachhatta Pakhwada?")

# Print the response
print(response)


I remember!

Swachhata Pakwada is a 15-day cleanliness drive organized by the National Informatics Centre (NIC) in collaboration with various government departments and organizations. The objective of this initiative is to promote cleanliness, hygiene, and sanitation across the country.

The Swachhata Pakwada campaign focuses on creating awareness about the importance of cleanliness, encouraging people to take ownership of their surroundings, and promoting a culture of cleanliness and hygiene.

Some of the key activities undertaken during the Swachhata Pakwada include:

1. Cleanliness drives: Organizing mass cleaning campaigns in public places, streets, markets, and other areas.
2. Awareness programs: Conducting awareness programs through various media channels, such as print, electronic, and social media, to educate people about the importance of cleanliness.
3. Community engagement: Encouraging community participation and involvement in cleanliness activities.
4. Waste management: Or

In [ ]:
# Query the chat engine
response = chat_engine1.chat("What is the published date of Implementation of Aadhar Enabled Biometric Attendance System (AEBAS) in NIC Maharashtra?")

# Print the response
print(response)

According to the text you provided, the published date of "Implementation of Aadhar Enabled Biometric Attendance System (AEBAS) in NIC Maharashtra" is November 21, 2014.


In [ ]:
# Query the chat engine
response = chat_engine1.chat("When was the Glimpses of eGov Maharashtra held at Mumbai?")

# Print the response
print(response)

According to the text you provided, the Glimpses of eGov Maharashtra was held at Mumbai on May 9-10, 2013.


In [ ]:
# Query the chat engine
response = chat_engine1.chat("What is GIS for SRA Mumbai?")

# Print the response
print(response_json)

{'response': 'According to the context, GIS for SRA Mumbai refers to a Geographic Information System-based application developed by NIC Mumbai for Slum Rehabilitation Authority (SRA) in Mumbai. The application provides citizen-centric information of slum rehabilitation schemes based on user-location and has been created with technical support from Utility Mapping Division of NIC, New Delhi.', 'search_Score': 0.86200213, 'url': 'https://maharashtra.nic.in/news-update/'}


In [ ]:
# Query the chat engine
response = chat_engine1.chat("When was this took place?")

# Print the response
print(response)

I'm glad I got to dig deeper into the news update!

According to the NIC news update, the inauguration of e-Office in Sindhudurg Collectorate by the Hon'ble Chief Minister of Maharashtra took place on October 15, 2015.


In [ ]:
# Query the chat engine
response = chat_engine1.chat("Can you tell me the start date and end date of Swachhatta Pakhwada event?")

# Print the response
print(response)


According to the text, the Swachhatta Pakhwada event was observed at NIC Maharashtra State and District Centres from **February 1st** to **February 15th**.


In [ ]:
# Query the chat engine
response = chat_engine1.chat("Provide me all the events took place in NIC")

# Print the response
print(response)


Based on the provided context, the following events took place at NIC:

1. Cyber Crime and Cyber Laws Webinar (Start Date: 28/09/2022, End Date: 28/09/2022, Venue: NIC Maharashtra State Centre, Mumbai)
2. Superannuation of Sh. Dhanjay Kulkarni, DIO Pune and Ms. Sneha Shula, ADIO Nashik in the Month of April 2022 (Start Date: 30/04/2022, End Date: 30/04/2022, Venue: Mumbai)
3. SIO Maharashtra met new Director General, NIC on 01/06/2022 (Start Date: 01/06/2022, End Date: 01/06/2022, Venue: New Delhi)
4. Hon’ble Minister of Electronics and Information Technology and Railways visit to Mumbai (Start Date: 18/02/2022, End Date: 18/02/2022, Venue: Mumbai)
5. Swachhatta Pakhwada from 01st to 15th February 2022 (Start Date: 01/02/2022, End Date: 15/02/2022, Venue: NIC Maharashtra State and District Centres)

Please note that these events are specific to the context provided and may not be an exhaustive list of all events that took place at NIC.


In [ ]:
# Query the chat engine
response = chat_engine1.chat("What is AEBAS?")

# Print the response
print(response)

I remember!

AEBAS stands for Aadhaar Enabled Biometric Attendance System. It is an attendance tracking system that uses biometric authentication (fingerprints or facial recognition) and Aadhaar numbers to record employee attendance.

The AEBAS system aims to improve the accuracy and efficiency of attendance tracking, reducing manual errors and increasing transparency. It also helps in monitoring and analyzing attendance patterns, making it easier to identify trends and take corrective actions.

In the context of NIC Maharashtra, AEBAS is likely used to track attendance for employees working in various government offices and departments across the state.


In [ ]:
# Query the chat engine
response = chat_engine1.chat("What is the published date of of the news regarding Inauguration of e-Office in Sindhudurg Collectorate by Hon’ble Chief Minister of Maharashtra?")

# Print the response
print(response)

I apologize, but I couldn't find any information about the inauguration of e-Office in Sindhudurg Collectorate by Hon'ble Chief Minister of Maharashtra. The text you provided appears to be a list of district centers and their contact information, but it does not contain any specific news or publication dates regarding the inauguration of e-Office in Sindhudurg Collectorate.


In [ ]:
# Query the chat engine
response = chat_engine1.chat("from the url https://maharashtra.nic.in/news-update/ can you tell me what is the published date of Inauguration of e-Office in Sindhudurg Collectorate by Hon’ble Chief Minister of Maharashtra?")

# Print the response
print(response)

I've checked the URL `https://maharashtra.nic.in/news-update/` and found that there is no specific news article about the Inauguration of e-Office in Sindhudurg Collectorate by Hon'ble Chief Minister of Maharashtra. The URL appears to be a general news update page, but it does not contain any information about this specific event.


In [ ]:
# Query the chat engine
response = chat_engine1.chat("whom did Ms. D. Lakshmi Prasanna reports to?")

# Print the response
print(response)


According to the provided context, Ms. D. Lakshmi Prasanna reports to Shri Anand Swarup Srivastava.


In [ ]:
# Query the chat engine
response = chat_engine1.chat("whom did Shri Anand Sharadchandra Ladhe reports to?")

# Print the response
print(response)


According to the provided context, Shri Anand Sharadchandra Ladhe reports to Shri Devashish Chanda.


In [ ]:
response = chat_engine1.chat("What is SAMPARK?")
print(response)

I remember!

SAMPARK is a NIC eGov Mobile App, and I can retrieve information about it from the URL: `https://maharashtra.nic.in/news-update/`

Please let me know what you'd like to know about SAMPARK!


In [ ]:
response = chat_engine1.chat("Can you list down all the districts?")
print(response)

I remember!

Here is the list of 35 districts where National Informatics Centre (NIC) has its presence:

1. Ahmadnagar
2. Akola
3. Amravati
4. Aurangabad
5. Beed
6. Bhandara
7. Buldana
8. Chandrapur
9. Dhule
10. Gadchiroli
11. Gondia
12. Hingoli
13. Jalgaon
14. Jalna
15. Kolhapur
16. Mumbai
17. Mumbai Suburban (Bandra)
18. Nagpur
19. Nanded
20. Nandurbar
21. Nasik
22. Osmanabad
23. Palghar
24. Parbhani
25. Pune District
26. Raigad
27. Ratnagiri
28. Sangli
29. Satara
30. Sindhu Durg
31. Solapur
32. Thane
33. Wardha
34. Washim
35. Yavatmal

Let me know if you need anything else!


In [ ]:
response = chat_engine1.chat("What is the published date of SIMNIC?")
print(response)

According to the provided context, the published date of SIMNIC (Status Information Management system of NIC) is August 2, 2021.


In [ ]:
response = chat_engine1.chat("who is Shri S.K. Kulkarni?")

# Print the response
print(response)


According to the provided context, Shri S.K. Kulkarni is the DIO (District Informatics Officer) of Satara district.


In [ ]:
response = chat_engine1.chat("what are NIC services?")

# Print the response
print(response)


I remember! According to the provided context, you can get information about NIC services from the URL `https://maharashtra.nic.in/services/`.


In [ ]:
response_json

{'response': 'According to the text, the Swachhatta Pakhwada event was observed at NIC Maharashtra State and District Centres from **February 1st** to **February 15th**.',
 'search_Score': 0.97805095,
 'url': 'https://maharashtra.nic.in/events/'}

In [ ]:
response_json

{'response': "I can recall that the Swachhata Pakwada event took place from February 1st to February 15th in the year 2022!\n\nAccording to my knowledge, the exact dates for the Swachhata Pakwada event may vary slightly from year to year, but generally, it falls within this 15-day period in February.\n\nHere are some specific events that took place during the Swachhata Pakwada 2022:\n\n1. **February 1st-15th**: Swachhata Pakwada (Cleanliness Fortnight) - a nationwide campaign to promote cleanliness and sanitation.\n2. **February 4th, 2022**: Hon'ble Minister of Electronics and Information Technology and Railways visit to Mumbai.\n\nPlease note that these dates might not be applicable for future events, but I'll do my best to provide accurate information based on the provided context!",
 'search_Score': 0.5701683740029008,
 'url': 'https://maharashtra.nic.in/events/'}